In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

 98% 1.18G/1.21G [00:07<00:00, 251MB/s]
100% 1.21G/1.21G [00:07<00:00, 168MB/s]


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gtzan-dataset-music-genre-classification.zip')
zip_ref.extractall('/content/Data')
zip_ref.close()

In [1]:
import os
Root = "/content/Data/Data"
os.chdir(Root)

In [2]:
import pandas as pd
import numpy as np
import os
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
audio_dataset_path='/content/Data/Data/genres_original'
metadata=pd.read_csv('/content/Data/Data/features_30_sec.csv')
metadata.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


In [4]:
def features_extractor(file):
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
  mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
  mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)

  return mfccs_scaled_features

# from sklearn.preprocessing import StandardScaler

# # ... (your existing code)

# # Modify features_extractor function
# def features_extractor(file):
#     audio, sample_rate = librosa.load(file, res_type='kaiser_fast')
#     mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
#     mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
#     return mfccs_scaled_features

# # Normalize input features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [66]:
# metadata.drop(labels=552, axis=0, inplace=True)

In [5]:
from tqdm import tqdm

extracted_features=[]
for index_num, row in tqdm(metadata.iterrows()):
  try:
    final_class_labels=row['label']
    file_name = os.path.join(os.path.abspath(audio_dataset_path), final_class_labels+'/',str(row['filename']))
    data=features_extractor(file_name)
    extracted_features.append([data, final_class_labels])
  except Exception as e:
    print(f"Error: {e}")
    continue

553it [00:45, 13.99it/s]<ipython-input-4-761b85e0e388>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
557it [00:46, 11.77it/s]

Error: 


1000it [01:23, 11.96it/s]


In [6]:
extracted_features_df = pd.DataFrame(extracted_features, columns=['featues', 'class'])
extracted_features_df.head()

,featues,class
0,"[-113.59882, 121.57067, -19.162262, 42.36394, ...",blues
1,"[-207.52383, 123.98514, 8.94702, 35.86715, 2.9...",blues
2,"[-90.757164, 140.44087, -29.084547, 31.686693,...",blues
3,"[-199.57513, 150.0861, 5.663404, 26.855282, 1....",blues
4,"[-160.35417, 126.20948, -35.581394, 22.139256,...",blues


In [7]:
extracted_features_df['class'].value_counts()

blues        100
classical    100
country      100
disco        100
hiphop       100
metal        100
pop          100
reggae       100
rock         100
jazz          99
Name: class, dtype: int64

In [8]:
X = np.array(extracted_features_df['featues'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [9]:
X.shape, y.shape

((999, 40), (999,))

In [10]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

In [11]:
y.shape

(999, 10)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(799, 40)
(200, 40)
(799, 10)
(200, 10)


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow. keras.optimizers import Adam
from sklearn import metrics

In [15]:
num_labels = y.shape[1]

In [16]:
model=Sequential()
model.add(Dense (1024, input_shape=(40,), activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(512, activation="relu"))
model.add(Dropout (0.3))
model.add(Dense (256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense (128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense (64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense (32, activation="relu"))
model.add(Dropout(0.3))

###final layer
model.add (Dense(num_labels, activation="softmax"))

# from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D


# model = Sequential()
# model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(64, kernel_size=3, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(GlobalAveragePooling1D())
# model.add(Dense(num_labels, activation='softmax'))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              41984     
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               3

In [18]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [19]:
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)

In [36]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from datetime import datetime

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath=f' saved_models/audio_classification_{current_time} .hdf5',
                                verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


start = datetime.now()
history = model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
                    validation_data=(X_test, y_test), callbacks=[checkpointer, early_stopping])

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
20/25 [=======================>......] - ETA: 0s - loss: 0.1502 - accuracy: 0.9578
Epoch 1: val_loss improved from inf to 3.56917, saving model to  saved_models/audio_classification_16:45:02 .hdf5
25/25 [==============================] - 0s 14ms/step - loss: 0.1496 - accuracy: 0.9549 - val_loss: 3.5692 - val_accuracy: 0.6550
Epoch 2/200
21/25 [========================>.....] - ETA: 0s - loss: 0.2054 - accuracy: 0.9628

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss did not improve from 3.56917
25/25 [==============================] - 0s 7ms/step - loss: 0.2316 - accuracy: 0.9549 - val_loss: 3.7645 - val_accuracy: 0.6250
Epoch 3/200
18/25 [====================>.........] - ETA: 0s - loss: 0.2549 - accuracy: 0.9392
Epoch 3: val_loss improved from 3.56917 to 3.52571, saving model to  saved_models/audio_classification_16:45:02 .hdf5
25/25 [==============================] - 0s 11ms/step - loss: 0.2310 - accuracy: 0.9462 - val_loss: 3.5257 - val_accuracy: 0.6400
Epoch 4/200
19/25 [=====================>........] - ETA: 0s - loss: 0.1949 - accuracy: 0.9556
Epoch 4: val_loss did not improve from 3.52571
25/25 [==============================] - 0s 8ms/step - loss: 0.1665 - accuracy: 0.9612 - val_loss: 3.8705 - val_accuracy: 0.6700
Epoch 5/200
21/25 [========================>.....] - ETA: 0s - loss: 0.2136 - accuracy: 0.9554
Epoch 5: val_loss improved from 3.52571 to 3.10572, saving model to  saved_models/audio_classification_16:45:02 .h

In [42]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/content/Data/Data/ saved_models/audio_classification_16:45:02 .hdf5')
print("Model loaded successfully.")

# Evaluate the loaded model on test data
loaded_model.evaluate(X_test, y_test, verbose=0)

Model loaded successfully.


[2.8976950645446777, 0.6600000262260437]

In [37]:
# model.evaluate(X_test, y_test, verbose=0)

[2.8976950645446777, 0.6600000262260437]

In [ ]:
pd.DataFrame(history.history).plot(figsize=(12,6))
plt.show()

In [43]:
predicted_probabilities = loaded_model.predict(X_test)
predicted_classes = np.argmax(predicted_probabilities, axis=1)

7/7 [==============================] - 0s 6ms/step


In [44]:
predicted_classes

array([7, 0, 0, 6, 9, 5, 3, 3, 0, 2, 6, 2, 4, 8, 9, 9, 4, 3, 3, 9, 9, 4,
       5, 9, 1, 3, 0, 4, 3, 3, 6, 9, 7, 4, 7, 7, 0, 8, 5, 4, 9, 5, 6, 0,
       2, 4, 7, 9, 3, 8, 3, 3, 4, 1, 8, 4, 4, 2, 8, 5, 8, 6, 5, 2, 7, 8,
       5, 8, 2, 4, 5, 8, 1, 7, 9, 4, 6, 4, 6, 2, 6, 4, 6, 5, 9, 9, 5, 4,
       2, 7, 9, 8, 8, 5, 2, 6, 9, 4, 5, 0, 1, 5, 6, 6, 7, 5, 1, 0, 5, 5,
       2, 1, 8, 4, 9, 0, 7, 8, 2, 5, 0, 3, 8, 6, 3, 2, 0, 4, 6, 9, 7, 8,
       3, 3, 4, 9, 8, 1, 7, 2, 8, 3, 4, 9, 4, 0, 0, 1, 0, 5, 3, 9, 7, 8,
       8, 8, 8, 3, 9, 4, 1, 5, 1, 5, 7, 9, 8, 7, 1, 0, 2, 8, 4, 3, 4, 1,
       7, 4, 5, 3, 2, 3, 3, 3, 3, 5, 9, 3, 8, 0, 4, 6, 7, 8, 6, 1, 2, 8,
       3, 8])

In [46]:
filename="/content/Data/Data/genres_original/rock/rock.00000.wav"
audio, sample_rate= librosa.load(filename, res_type=' kaiser_fast')
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np. mean (mfccs_features.T, axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1, -1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)

predicted_probabilities = model.predict(mfccs_scaled_features)
predicted_classes = np.argmax(predicted_probabilities, axis=1)
print(predicted_classes)
prediction_class = labelencoder.inverse_transform(predicted_classes)
prediction_class

[-1.16626266e+02  1.09252632e+02 -2.31637516e+01  4.06794815e+01
 -1.18888369e+01  2.31529446e+01 -1.54092770e+01  1.51249895e+01
 -1.35225782e+01  1.26958866e+01 -1.08715563e+01  8.49799061e+00
 -1.15775967e+01  2.19769740e+00 -6.18050623e+00  1.98997986e+00
 -7.95588875e+00  2.08338356e+00 -2.39291382e+00 -1.54179323e+00
 -2.62624383e+00 -4.30706561e-01 -3.03475261e+00 -8.14875305e-01
  1.09406956e-01 -2.70748091e+00 -1.16246223e+00 -6.56219339e+00
  8.08540881e-01 -5.98537397e+00 -6.39179766e-01  1.83361018e+00
 -1.91109502e+00 -2.85637426e+00 -2.22769475e+00 -7.77185336e-02
 -3.61104631e+00 -1.35419798e+00  2.51583785e-01 -2.90771270e+00]
[[-1.16626266e+02  1.09252632e+02 -2.31637516e+01  4.06794815e+01
  -1.18888369e+01  2.31529446e+01 -1.54092770e+01  1.51249895e+01
  -1.35225782e+01  1.26958866e+01 -1.08715563e+01  8.49799061e+00
  -1.15775967e+01  2.19769740e+00 -6.18050623e+00  1.98997986e+00
  -7.95588875e+00  2.08338356e+00 -2.39291382e+00 -1.54179323e+00
  -2.62624383e+00 -

array(['rock'], dtype='<U9')